# Init

## Import

In [ ]:
import seaborn as sns
import amplitudes
import generate_plots as gp
import matplotlib.pylab as plt
from collections import OrderedDict
import copy

import scipy.interpolate as itp
import scipy.optimize as opt
import matplotlib.patches as patches

from io import StringIO
import pandas as pd
import numpy as np

# import fine_tuning

# import myCollections.non_linear_fit2 as nlf

import re

from sympy import S
from sympy.solvers import solve

# reload(gp)
# reload(amplitudes)

# sns.set(font="Source Sans Pro", style="whitegrid", font_scale=1.8)
sns.set(font="Source Sans Pro", style="ticks", font_scale=1.4)
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})

import logging

## Defs

In [ ]:
logger = logging.getLogger("generate_plots")
logger.setLevel(logging.WARNING)

a4_width = 8.3 * 0.9 * 1.5
tex_nuc = {
    "deut": "$^2H$",
    "h3": "$^3H$",
    "he3": "$^3He$",
}

# Regular Plots

## Setup

In [ ]:
pars = gp.parameters(normalized=True, self_normalized=True, isoscalar_only=True)

# Eliminiate iv contributions
for key, val in pars.facts.items():
    pars.facts[key] = val.subs({"delta_mN": 0})
    print(key)
    display(pars.facts[key])

nucleons = ["deut", "h3", "he3"]
nucleons = ["deut", "he3"]

amps = {}
labels = {}
cont = {}
ops = {}

In [ ]:
for nucleon in nucleons:
    ops[nucleon] = operators = gp.get_operators(nucleon, parameters=pars)
    ams = gp.get_amplitudes(operators, parameters=pars)

    amps[nucleon] = list(
        ams.query("sm == 'q' and (type == 'amp' or type == '2B' or type=='1B') ")["val"]
    )
    labels[nucleon] = list(
        ams.query("sm == 'q' and (type == 'amp' or type == '2B' or type=='1B') ")[
            "label"
        ]
    )
    # amps[nucleon]   = list(ams.query("sm == 'g' and (type == 'amp' or type == 'NLO') ")["val"])
    # labels[nucleon] = list(ams.query("sm == 'g' and (type == 'amp' or type == 'NLO') ")["label"])
    cont[nucleon] = amplitudes.OperatorPlotContainer(
        amps[nucleon], estimate_type=2
    )  # , modify_Q=1.)

## Fitting Helm Form Factors

In [ ]:
fit_waves = {
    "he3": "ich-n4lo-cut=2-no3nf-E=search-cdep-empot",
    "deut": "ich-n4lo-cut=2+empot",
    "h3": "ich-n4lo-cut=2-no3nf-E=search-cdep-empot",
}

A = {
    "he3": 3,
    "h3": 3,
    "deut": 2,
}

hbarc = amplitudes.phys_consts.hbarc.MeV

In [ ]:
pars = {}
perr = {}
chi = {}
HFF = {}

for nucleon in amps.keys():

    wave_query = fit_waves[nucleon]
    c = cont[nucleon]

    wave_data = c.chiral_estimates[0].query("Order=='n2lo' and Cutoff == 'c2'")
    data = wave_data["amp"].values
    qMeV = wave_data["q"].values
    sigma = wave_data["d_amp"].values

    q = qMeV / hbarc

    def get_HFF(qin, a, s):
        hff = amplitudes.HelmFormFactor(A=A[nucleon], a=a, s=s)
        return hff.amp(qin) ** 2

    pars[nucleon], pcov = opt.curve_fit(
        get_HFF, q, data, p0=[0.5, 0.9], bounds=[(0.0, 0.1), (2.0, 1.5)], sigma=sigma
    )
    perr[nucleon] = np.sqrt(np.diag(pcov))

    print(nucleon, pars[nucleon], perr[nucleon])

    HFF[nucleon] = amplitudes.HelmFormFactor(
        A=A[nucleon], a=pars[nucleon][0], s=pars[nucleon][1]
    )

    plt.plot(qMeV, data, "o", label="AV18" + nucleon)

    q_itp = np.linspace(q.min(), q.max(), 100)

    plt.plot(q_itp * hbarc, HFF[nucleon].amp(q_itp) ** 2, "-", label="HFF" + nucleon)

    plt.legend()

plt.show()

chi

## Get Tables

### Delta_X

In [ ]:
nucleon = "deut"

In [ ]:
op = ops[nucleon]["scalar"]

f = op.frame.query("q == 0 and xi==0").drop(["xi", "mxi"], axis=1)
f = op.get_chiral_frame(frame=f, tex=False).rename(columns={"mat": "amp"})
f = f[[u"q", u"amp", u"Cutoff", u"Order"]]


zero = f.set_index(["Order", "Cutoff"]).drop("q", axis=1)
zero = zero.to_dict("index")

In [ ]:
op = ops[nucleon]["2-pion-q"]

f = op.frame.query("q == 0 and xi==0").drop(["xi", "mxi"], axis=1)
f = op.get_chiral_frame(frame=f, tex=False).rename(columns={"mat": "amp"})
f = f[[u"q", u"amp", u"Cutoff", u"Order"]]

# normalize
f.loc[:, "amp"] = f.apply(
    lambda row: row["amp"] / zero[(row["Order"], row["Cutoff"])]["amp"], axis=1
)

est = amplitudes.ChiralErrorEstimate(f, modify_Q=1.0)

In [ ]:
c = cont[nucleon] 
f = c.delta_X[1]
f = est.del_x_frame    

print(labels["deut"][1])

f = f.query("q == 0")
f.loc[:,"label"] = f.apply(
    lambda rows: str(int(rows["nu1"])) + "-" +  str(int(rows["nu2"])), 
    axis=1
)

f1 = f.copy()
f2 = f.copy()
f1.loc[:,"Q"] = f.loc[:,"delta_X"]
f2.loc[:,"Q"] = 0
f3 = pd.concat([f1,f2]*10)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), dpi=500)

sns.boxplot(
    x="label",
    y="Q",
    hue="cutoff",
    data=f3,
    orient="v",
    linewidth=0.1,
)

plt.ylabel(r"$\Delta X^{(\nu_1 , \nu_2)}(3Q_0)$")
plt.xlabel(r"$\nu_1 - \nu_2$")

# plt.title(labels["deut"][1] + "  at  $ q = 0 \mathrm{ MeV}$\n")
# plt.title("Deut 2$\pi$ OP at $q=0$ MeV")

plt.legend(loc="best", frameon=True)

ax.set_yscale("log")

plt.show()

In [ ]:
fig.savefig("delta_X_op_2pi.pdf", bbox_inches="tight")

### Values

#### Regular

In [ ]:
def get_phen_wave_name(wave):
    if wave == "av18-empot-no3nf-E=search-cdep-j12max=6":
        return "AV18", "no TNF"
    elif wave == "av18+empot":
        return "AV18", "no TNF"
    elif wave == "av18-empot-urbanaIXp16n5-E=search-cdep-j12max=6":
        return "AV18", "TNF"
    elif wave == "cdb+empot":
        return "CDB", "no TNF"
    elif wave == "cdb-empot-no3nf-E=search-cdep-j12max=6":
        return "CDB", "no TNF"
    elif wave == "cdb-empot-tm99l4469p16n5-cdb2000-E=search-cdep-j12max=6":
        return "CDB", "TNF"
    elif wave == "nijm93-noempot":
        return "NIJM", "no TNF"
    else:
        print(wave)

In [ ]:
amp_title = r"$ \left\langle \psi | \hat{J_q^{NN}} (q \vec{e}_z) | \psi \right\rangle $"

In [ ]:
order_pattern = {
    "-lo-": r"LO   ",
    "-nlo-": r"NLO",
    "-n2lo-": r"N$^2$LO",
    "-n3lo-": r"N$^3$LO",
    "-n4lo-": r"N$^4$LO",
}


def wave2tex(row):
    wave = row["wave"]
    # is chiral
    if re.findall("^ich", wave):
        order = order_pattern[re.findall("-[n1-9]*lo-", wave)[0]]
        cutoff = "$\Lambda_" + re.findall("-cut=([1-9])+", wave)[0] + "$"
        return "$\chi$ " + order, cutoff
    else:
        return get_phen_wave_name(wave)

In [ ]:
frames = []

for nucleon in nucleons:
    op = ops[nucleon]["scalar"]
    f = (
        op.frame.query("q == 0 and xi==0")
        .drop(["xi", "mxi", "q"], axis=1)
        .set_index("wave")
    )
    f.sort_index(inplace=True)
    norm = f.values.flatten()

    title = tex_nuc[nucleon]  # +" "+  r"$\left\langle \mathcal{O} \right\rangle$"
    op = ops[nucleon]["2-pion-q"]
    f = (
        op.frame.query("q == 0 and xi==0")
        .drop(["xi", "mxi", "q"], axis=1)
        .set_index("wave")
    )
    f.sort_index(inplace=True)
    f.loc[:, title] = f.loc[:, "mat"] / norm
    f.reset_index(inplace=True)

    wave = np.array(list(map(list, f.apply(wave2tex, axis=1).values)))
    f["Wave"] = wave.T[0]
    f["Detail"] = wave.T[1]

    f.set_index(["Wave", "Detail"], inplace=True)
    f.drop(["wave", "mat"], inplace=True, axis=1)

    frames.append(f)

table = pd.concat(frames, axis=1)
table.sort_index(inplace=True)
print(
    table.to_latex(
        na_rep="/",
        float_format="%1.2e",
        escape=False,
        col_space=8,
        # header=False
    )
)

## 2B - D Prob

In [ ]:
texstr = r"""
NN interaction &     h2 &     h3 &    he3 
\hline\hline
AV18 + Urb IXF          &         / &  2.62e-03 &  2.52e-03 
AV18                    &  1.58e-02 &  2.62e-03 &  2.83e-03 
CD-Bonn + TM            &         / &  1.38e-03 &  1.51e-03 
CD-Bonn                 &  1.17e-02 &  1.47e-03 &  1.62e-03 
NIJM                    &  1.57e-02 &         / &         / 
\hline
$Q^{0}$   $\Lambda_{1}$ &  2.28e-03 & -1.46e-03 & -1.45e-03 
$Q^{0}$   $\Lambda_{2}$ & -5.95e-04 & -1.77e-03 & -1.77e-03 
$Q^{0}$   $\Lambda_{3}$ & -2.66e-03 & -2.01e-03 & -2.02e-03 
$Q^{0}$   $\Lambda_{4}$ & -4.12e-03 & -2.20e-03 & -2.22e-03 
$Q^{0}$   $\Lambda_{5}$ & -5.13e-03 & -2.36e-03 & -2.38e-03 
\hline 
$Q^{2}$   $\Lambda_{1}$ &  1.17e-02 &  1.48e-03 &  1.62e-03 
$Q^{2}$   $\Lambda_{2}$ &  8.80e-03 &  7.08e-04 &  8.18e-04 
$Q^{2}$   $\Lambda_{3}$ &  6.09e-03 &  2.72e-05 &  1.08e-04 
$Q^{2}$   $\Lambda_{4}$ &  3.70e-03 & -5.47e-04 & -4.91e-04 
$Q^{2}$   $\Lambda_{5}$ &  1.65e-03 & -1.01e-03 & -9.78e-04 
\hline 
$Q^{3}$   $\Lambda_{1}$ &  1.10e-02 &  1.37e-03 &  1.51e-03 
$Q^{3}$   $\Lambda_{2}$ &  8.59e-03 &  7.25e-04 &  8.39e-04 
$Q^{3}$   $\Lambda_{3}$ &  6.17e-03 &  1.06e-04 &  1.93e-04 
$Q^{3}$   $\Lambda_{4}$ &  3.97e-03 & -4.39e-04 & -3.75e-04 
$Q^{3}$   $\Lambda_{5}$ &  2.04e-03 & -8.98e-04 & -8.55e-04 
\hline 
$Q^{4}$   $\Lambda_{1}$ &  7.31e-03 &  7.54e-04 &  8.61e-04 
$Q^{4}$   $\Lambda_{2}$ &  1.09e-02 &  1.66e-03 &  1.81e-03 
$Q^{4}$   $\Lambda_{3}$ &  1.29e-02 &  2.21e-03 &  2.38e-03 
$Q^{4}$   $\Lambda_{4}$ &  1.37e-02 &  2.44e-03 &  2.62e-03 
$Q^{4}$   $\Lambda_{5}$ &  1.38e-02 &  2.50e-03 &  2.69e-03 
\hline 
$Q^{5}$   $\Lambda_{1}$ &  1.06e-02 &  1.62e-03 &  1.77e-03 
$Q^{5}$   $\Lambda_{2}$ &  1.13e-02 &  1.77e-03 &  1.92e-03 
$Q^{5}$   $\Lambda_{3}$ &  1.18e-02 &  1.91e-03 &  2.07e-03 
$Q^{5}$   $\Lambda_{4}$ &  1.25e-02 &  2.10e-03 &  2.27e-03 
$Q^{5}$   $\Lambda_{5}$ &  1.27e-02 &  2.20e-03 &  2.38e-03 
"""

repl = {
    " ": "",
    r"[${}]": "",
}


for pattern, subs in repl.items():
    texstr = re.sub(pattern, subs, texstr)

fle = StringIO(texstr)


op_frame = pd.read_csv(fle, sep=r"&").dropna().set_index(u"NNinteraction")


# op_frame

In [ ]:
texstr = r"""
NN interaction           & $E_{d}$ & $\langle T \rangle $ &$\langle V \rangle$& $r_{d}$ &   $Q_{d}$ & $ P_{D} $ & $A_{S}$  & $\eta$
\hline \hline
AV18                         & -2.225    &  19.81                      &  -22.04                  &   1.967  &      0.270 &        5.76   & 0.884       &  0.0252   
CD-Bonn                   & -2.223    &  15.60                      & -17.83                   &   1.966  &      0.270 &        4.85   & 0.884       &  0.0258 
\hline
$Q^{0}$   $\Lambda_{1}$  & -1.989   & 14.26   &  -16.25   &   1.997   &   0.245   &    3.27     &     0.825    &    0.0219    
$Q^{0}$   $\Lambda_{2}$  & -2.023   & 13.29   &  -15.32   &   1.990   &   0.230   &    2.54     &     0.833    &    0.0212    
$Q^{0}$   $\Lambda_{3}$  & -2.083   & 12.47   &  -14.55   &   1.979   &   0.215   &    1.97     &     0.849    &    0.0205    
$Q^{0}$   $\Lambda_{4}$  & -2.167   & 11.76   &  -13.92   &   1.965   &   0.199   &    1.53     &     0.870    &    0.0198    
$Q^{0}$   $\Lambda_{5}$  & -2.272   & 11.15   &  -13.42   &   1.950   &   0.183   &    1.18     &     0.897    &    0.0192    
\hline 
$Q^{2}$   $\Lambda_{1}$  & -2.191   & 15.40   &  -17.59   &   1.970   &   0.275   &    5.23     &     0.875    &    0.0256    
$Q^{2}$   $\Lambda_{2}$  & -2.199   & 14.25   &  -16.45   &   1.968   &   0.273   &    4.73     &     0.877    &    0.0256    
$Q^{2}$   $\Lambda_{3}$  & -2.206   & 13.49   &  -15.69   &   1.967   &   0.271   &    4.24     &     0.879    &    0.0257    
$Q^{2}$   $\Lambda_{4}$  & -2.211   & 12.92   &  -15.14   &   1.965   &   0.269   &    3.77     &     0.881    &    0.0258    
$Q^{2}$   $\Lambda_{5}$  & -2.213   & 12.48   &  -14.69   &   1.965   &   0.267   &    3.35     &     0.881    &    0.0259    
\hline 
$Q^{3}$   $\Lambda_{1}$  & -2.228   & 14.94   &  -17.16   &   1.967   &   0.271   &    4.87     &     0.886    &    0.0254    
$Q^{3}$   $\Lambda_{2}$  & -2.231   & 13.85   &  -16.08   &   1.966   &   0.270   &    4.50     &     0.886    &    0.0256    
$Q^{3}$   $\Lambda_{3}$  & -2.235   & 13.17   &  -15.40   &   1.964   &   0.270   &    4.12     &     0.888    &    0.0258    
$Q^{3}$   $\Lambda_{4}$  & -2.237   & 12.69   &  -14.93   &   1.964   &   0.269   &    3.75     &     0.888    &    0.0260    
$Q^{3}$   $\Lambda_{5}$  & -2.235   & 12.32   &  -14.55   &   1.963   &   0.269   &    3.40     &     0.887    &    0.0263    
\hline 
$Q^{4}$   $\Lambda_{1}$  & -2.223   & 23.33   &  -25.55   &   1.970   &   0.268   &    3.78     &     0.884    &    0.0255    
$Q^{4}$   $\Lambda_{2}$  & -2.223   & 21.58   &  -23.80   &   1.972   &   0.271   &    4.19     &     0.884    &    0.0255    
$Q^{4}$   $\Lambda_{3}$  & -2.223   & 19.63   &  -21.85   &   1.975   &   0.275   &    4.77     &     0.885    &    0.0256    
$Q^{4}$   $\Lambda_{4}$  & -2.223   & 17.71   &  -19.94   &   1.979   &   0.279   &    5.21     &     0.885    &    0.0256    
$Q^{4}$   $\Lambda_{5}$  & -2.223   & 16.13   &  -18.35   &   1.982   &   0.283   &    5.58     &     0.885    &    0.0256    
\hline 
$Q^{5}$   $\Lambda_{1}$  & -2.223   & 20.64   &  -22.86   &   1.970   &   0.271   &    4.28     &     0.884    &    0.0256    
$Q^{5}$   $\Lambda_{2}$  & -2.223   & 18.89   &  -21.12   &   1.972   &   0.271   &    4.29     &     0.884    &    0.0256    
$Q^{5}$   $\Lambda_{3}$  & -2.223   & 17.48   &  -19.70   &   1.974   &   0.272   &    4.40     &     0.884    &    0.0255    
$Q^{5}$   $\Lambda_{4}$  & -2.223   & 16.29   &  -18.51   &   1.978   &   0.276   &    4.74     &     0.885    &    0.0256    
$Q^{5}$   $\Lambda_{5}$  & -2.223   & 15.27   &  -17.50   &   1.981   &   0.280   &    5.12     &     0.885    &    0.0256    

"""

for pattern, subs in repl.items():
    texstr = re.sub(pattern, subs, texstr)

fle = StringIO(texstr)


d_frame = pd.read_csv(fle, sep=r"&").dropna().set_index(u"NNinteraction")

nf_d = pd.concat([op_frame, d_frame], axis=1)[["h2", "P_D"]].dropna()

nf_d.loc[:, "psi"] = "$^2H$"
nf_d.reset_index(inplace=True)
nf_d.columns = [u"wave", u"O", u"PD", u"psi"]
# nf_d

In [ ]:
texstr = r"""
NN interaction           & $E(^{3}{\rm He})$ & $\langle T \rangle $ &$\langle V \rangle$& $r_{p}$ & $r_{n}$ &  $r_{NN}$ &   $P_{S}$ & $ P_{P} $ & $ P_{D} $  & $ \langle \Psi  |   \Psi \rangle $ 
\hline \hline
AV18                                  & -6.922  & 45.67   &  -52.59 &  1.871  &    1.677    &  3.127      &      91.47       &    0.065     &   8.465 &  0.998752 
AV18 + Urb IX                    &  -7.754  & 50.21   &  -57.97 &  1.770  &    1.601    &  2.966     &      90.63       &    0.131     &   9.241 &  0.998956 
CD-Bonn                            &  -7.264  & 36.77   &  -44.03 &  1.819   &   1.637    &  3.047     &      92.95       &    0.046     &   7.000 &  0.999505  
CD-Bonn + TM                   & -7.729   & 38.53  &  -46.26  &  1.767   &   1.598    &  2.964     &      92.41       &    0.090     &   7.498 &  0.999589  
\hline
$Q^{0}$   $\Lambda_{1}$  & -11.22   & 54.29   &  -65.51   & 1.327  &    1.258   &   2.259     &       95.31     &     0.031    &    4.659 & 0.999959  
$Q^{0}$   $\Lambda_{2}$  & -10.92   & 48.71   &  -59.63   & 1.367  &    1.289   &   2.324     &       96.65     &     0.015    &    3.333 & 0.999969  
$Q^{0}$   $\Lambda_{3}$  & -10.47   & 43.40   &  -53.87   & 1.424  &    1.334   &   2.416     &       97.58     &     0.008    &    2.410 & 0.999972  
$Q^{0}$   $\Lambda_{4}$  & -10.01   & 38.70   &  -48.70   & 1.489  &    1.388   &   2.522     &       98.23     &     0.004    &    1.763 & 0.999976  
$Q^{0}$   $\Lambda_{5}$  & -9.594   & 34.67   &  -44.27   & 1.558  &    1.444   &   2.634     &       98.69     &     0.002    &    1.304 & 0.999979  
\hline 
$Q^{2}$   $\Lambda_{1}$  & -7.315   & 36.61   &  -43.92   & 1.805  &    1.626   &   3.025     &       92.19     &     0.057    &    7.755 & 0.999781  
$Q^{2}$   $\Lambda_{2}$  & -7.481   & 33.96   &  -41.44   & 1.784  &    1.610   &   2.992     &       93.06     &     0.046    &    6.890 & 0.999881  
$Q^{2}$   $\Lambda_{3}$  & -7.638   & 32.75   &  -40.38   & 1.764  &    1.595   &   2.960     &       93.94     &     0.037    &    6.025 & 0.999924  
$Q^{2}$   $\Lambda_{4}$  & -7.804   & 32.21   &  -40.01   & 1.744  &    1.579   &   2.927     &       94.76     &     0.029    &    5.210 & 0.999946  
$Q^{2}$   $\Lambda_{5}$  & -7.969   & 31.96   &  -39.93   & 1.723  &    1.563   &   2.895     &       95.51     &     0.022    &    4.466 & 0.999959  
\hline 
$Q^{3}$   $\Lambda_{1}$  & -7.305   & 35.68   &  -42.99   & 1.821  &    1.639   &   3.050     &       92.99     &     0.045    &    6.966 & 0.999771  
$Q^{3}$   $\Lambda_{2}$  & -7.409   & 32.88   &  -40.29   & 1.806  &    1.628   &   3.027     &       93.61     &     0.039    &    6.357 & 0.999877  
$Q^{3}$   $\Lambda_{3}$  & -7.540   & 31.63   &  -39.17   & 1.788  &    1.614   &   2.999     &       94.26     &     0.032    &    5.713 & 0.999925  
$Q^{3}$   $\Lambda_{4}$  & -7.680   & 31.13   &  -38.81   & 1.769  &    1.599   &   2.969     &       94.90     &     0.027    &    5.078 & 0.999947  
$Q^{3}$   $\Lambda_{5}$  & -7.824   & 30.96   &  -38.79   & 1.749  &    1.584   &   2.937     &       95.51     &     0.022    &    4.470 & 0.999960  
\hline 
$Q^{4}$   $\Lambda_{1}$  & -6.865   & 51.81   &  -58.67   & 1.884  &    1.691   &   3.148     &       94.79     &     0.027    &    5.189 & 0.998593  
$Q^{4}$   $\Lambda_{2}$  & -6.871   & 50.74   &  -57.61   & 1.890  &    1.697   &   3.159     &       94.07     &     0.032    &    5.897 & 0.998904  
$Q^{4}$   $\Lambda_{3}$  & -6.832   & 46.94   &  -53.77   & 1.901  &    1.707   &   3.180     &       93.12     &     0.041    &    6.842 & 0.999219  
$Q^{4}$   $\Lambda_{4}$  & -6.812   & 42.31   &  -49.13   & 1.910  &    1.716   &   3.196     &       92.43     &     0.048    &    7.524 & 0.999481  
$Q^{4}$   $\Lambda_{5}$  & -6.779   & 38.21   &  -44.99   & 1.921  &    1.725   &   3.215     &       91.89     &     0.053    &    8.052 & 0.999661  
\hline 
$Q^{5}$   $\Lambda_{1}$  & -6.790   & 52.31   &  -59.10   & 1.898  &    1.701   &   3.170     &       93.96     &     0.033    &    6.005 & 0.998833  
$Q^{5}$   $\Lambda_{2}$  & -6.893   & 47.33   &  -54.22   & 1.886  &    1.693   &   3.154     &       93.92     &     0.033    &    6.046 & 0.999185  
$Q^{5}$   $\Lambda_{3}$  & -6.931   & 43.30   &  -50.23   & 1.885  &    1.695   &   3.155     &       93.71     &     0.035    &    6.255 & 0.999424  
$Q^{5}$   $\Lambda_{4}$  & -6.919   & 39.84   &  -46.76   & 1.893  &    1.702   &   3.169     &       93.17     &     0.040    &    6.786 & 0.999597  
$Q^{5}$   $\Lambda_{5}$  & -6.872   & 36.75   &  -43.62   & 1.906  &    1.714   &   3.192     &       92.60     &     0.046    &    7.351 & 0.999719  
\hline \hline
"""

for pattern, subs in repl.items():
    texstr = re.sub(pattern, subs, texstr)

fle = StringIO(texstr)


he_frame = pd.read_csv(fle, sep=r"&").dropna().set_index(u"NNinteraction")


he_frame

nf_he = pd.concat([op_frame, he_frame], axis=1)[["he3", "P_D"]].dropna()

nf_he.loc[:, "psi"] = "$^3He$"
nf_he.reset_index(inplace=True)
nf_he.columns = [u"wave", u"O", u"PD", u"psi"]
nf_he
cf = pd.concat([nf_d, nf_he], ignore_index=True)
cf.O = np.array(cf.O.values, dtype=float)

In [ ]:
from scipy import stats

c = cf.query("psi == '$^2H$'")
x = c["PD"]
y = c["O"]

slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

print(slope, intercept, r_value, p_value, std_err)

In [ ]:
colors = OrderedDict()
colors["pink"] = u"$\\chi$ LO"
colors["none"] = u"none"
colors["#FFEB8F"] = u"$\\chi$ NLO"
colors["#92D258"] = u"$\\chi$ N$^2$LO"
colors["#2A83FD"] = u"$\\chi$ N$^3$LO"
colors["#FB0007"] = u"$\\chi$ N$^4$LO"
colors["black"] = u"Phen"


def colormap(row):
    w = row["wave"]
    if w.startswith("Q^"):
        color = [el for el in colors.keys()][int(w[2])]
    else:
        color = "black"
    return color


def markermap(row):
    w = row["wave"]
    if w.startswith("Q^"):
        markerwidth = 5 + int(w[-1])
    else:
        markerwidth = 7
    return markerwidth


cf.loc[:, "color"] = cf.apply(colormap, axis=1)
cf.loc[:, "markerwidth"] = cf.apply(markermap, axis=1)

# cf

In [ ]:
sns.set(font="Source Sans Pro", style="ticks", font_scale=1.6)
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})

In [ ]:
# nf.columns[0:2] = ["$P_D$", r"$\left\langle \mathcal{O} _{2\pi} \right\rangle$"]

nf = cf.copy()  # .query("psi == '$^3He$'")

g = sns.jointplot("PD", "O", data=nf)

pearson = {
    u"$^2$H": "pearsonr = 0.96; p=4.9e-15",
    u"$^3$He": "pearsonr = 0.92; p=8.2e-12",
}

title = ""
for key, val in pearson.items():
    title += key + ": " + val + "\n"

markers = {u"$^2H$": "o", u"$^3He$": "s"}

labels = {
    u"$^2H$": u"$^2$H",
    u"$^3He$": u"$^3$He",
}


if 1:
    g.ax_joint.cla()

    for color in nf.color.unique():
        g.ax_joint.fill_between([np.NAN], [np.NAN], color=color, label=colors[color])

    for label in nf.psi.unique():
        g.ax_joint.plot(
            [np.NAN],
            [np.NAN],
            color="black",
            linestyle="none",
            marker=markers[label],
            label=labels[label],
        )

    sns.regplot(
        x="PD",
        y="O",
        data=cf.query("psi == '$^2H$'"),
        label="Fit $^2$H",
        scatter=False,
        ax=g.ax_joint,
    )

    sns.regplot(
        x="PD",
        y="O",
        data=cf.query("psi == '$^3He$'"),
        label="Fit $^3$He",
        scatter=False,
        ax=g.ax_joint,
        line_kws={"linestyle": ":"},
    )

    handles, labels = g.ax_joint.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))

    # Plot each individual point separately
    for n_psi, psi in enumerate(nf.psi.unique()):
        for n_wave, wave in enumerate(nf.wave.unique()):
            tf = nf.query("psi == @psi and wave == @wave ")
            for ind in tf.index:
                wave, O, PD, psi, color, marerwidth = tf.loc[ind].values
                g.ax_joint.plot(
                    PD,
                    O,
                    linestyle="None",
                    color=color,
                    marker=markers[psi],
                    markersize=marerwidth,
                )

    g.ax_joint.legend(
        by_label.values(),
        by_label.keys(),
        loc="upper left",
        bbox_to_anchor=(1.05, 1.08),
    )

In [ ]:
g.fig.set_figwidth(5)
g.fig.set_figheight(5)
g.fig.set_dpi(500)

g.ax_joint.xaxis.set_ticks_position("both")
g.ax_joint.yaxis.set_ticks_position("both")

# g.ax_marg_x.set_title(title)

g.ax_joint.set_ylim([-0.006, 0.015])
# g.ax_joint.set_yscale("log")

g.ax_marg_x.set_visible(False)
g.ax_marg_y.set_visible(False)

g.ax_joint.set_xlabel("$P_D$ in %")
g.ax_joint.set_ylabel(r"$\left\langle \hat {J}_{q^{(\mathrm{is})},2b} \right\rangle$")

sns.despine(ax=g.ax_joint, top=False, right=False)

g.fig

In [ ]:
g.savefig("D_wave_correlation.pdf", bbox_inches="tight")

In [ ]:
c = cf.query("psi == '$^2H$'")
x = c["PD"]
y = c["O"]

res = stats.linregress(x, y)

for label, el in zip(["slope", "intercept", "r_value", "p_value", "std_err"], res):
    print(label, ":{0:1.4e}".format(el))

In [ ]:
c = cf.query("psi == '$^3He$'")
x = c["PD"]
y = c["O"]

res = stats.linregress(x, y)

for label, el in zip(["slope", "intercept", "r_value", "p_value", "std_err"], res):
    print(label, ":{0:1.4e}".format(el))

### Form factors all orders

#### 2H

In [ ]:
a4_width = 8.3 * 0.9 * 1.5

nucleon = "deut"

fig = plt.figure(dpi=500, figsize=(a4_width, a4_width / 3.5 * 3))

cont[nucleon].plot_q_dep(
    fig,
)  # ylabels=labels[nucleon], legend=True)


plt.subplots_adjust(wspace=0.02, hspace=0.02 / 3.5 * 3)

plt.show()

# fig.savefig("err_deut.pdf", bbox_inches="tight")

#### 3He

In [ ]:
a4_width = 8.3 * 0.9 * 1.5

nucleon = "he3"

fig = plt.figure(dpi=500, figsize=(a4_width, a4_width / 3.5 * 3))

cont[nucleon].plot_q_dep(
    fig,
)  # ylabels=labels[nucleon], legend=True)


plt.subplots_adjust(wspace=0.02, hspace=0.02 / 3.5 * 3)

plt.show()

# fig.savefig("err_deut.pdf", bbox_inches="tight")

### Errors

#### Figure 2: 2H

In [ ]:
d_labels = [
    "$ | \\mathcal{{ {F} }}   _{{q^{{(\mathrm{is})}}}}^{\\left(0 \\right)} |^2 $",
    "$ | \\mathcal{{ {F} }}   _{{q^{{(\mathrm{is})}}}}^{\\left(0+1\\right)} |^2 $",
    "$ \\Delta^{(r)}  $ in %",
    "$ \\Delta^{(2b)} $ in %",
]
# d_labels = [
#    '$ | \\mathcal{{ {F} }}   _{{G}}^{\\left(0 \\right)} |^2 $',
#    '$ | \\mathcal{{ {F} }}   _{{G}}^{\\left(0 + 3\\right)} |^2 $',
#    '$ \\Delta^{(2b)} $ in %'
# ]

In [ ]:
a4_width = 8.3 * 0.9

nucleon = "deut"

fig = plt.figure(dpi=500, figsize=(a4_width, a4_width / 3.5 * 4))

cont[nucleon].plot_cut_dep(
    fig,
    ylabels=d_labels,
    legend=False,
    NQ=3,
    band=["n2lo", "n3lo", "n4lo"],
    band_type="average",
)
# cont[nucleon].plot_q_dep(fig, ylabels=labels[nucleon], legend=True)


plt.subplots_adjust(wspace=0.02, hspace=0.02 / 3.2 * 3)


axs = fig.get_axes()

ax = axs[1]
# ax.set_ylim([-8,1])

# ax = axs[6]
# ax.set_ylim([-20,30])

# leg = axs[8].get_legend()
# leg.remove()

handles, plt_labels = ax.get_legend_handles_labels()
by_label = dict(zip(plt_labels, handles))
by_label_sorted = OrderedDict()
for key in [u"NLO", u"N$^{2}$LO", u"N$^{3}$LO", u"N$^{4}$LO"]:
    by_label_sorted[key] = by_label[key]

ax = axs[0]
leg = ax.legend(
    by_label_sorted.values(),
    by_label_sorted.keys(),
    numpoints=1,
    loc="best",
    frameon=True,
)

ax = axs[10]
ax.set_ylim([-12, 17])
if nucleon == "he3":
    ax.set_ylim([-5, 5])

ax = axs[7]
ax.set_ylim([-8, 1])

for ax in axs:
    ytick_labels = ax.get_yticklabels()
    if ytick_labels:
        ax.get_yticklabels()[-1].set_visible(False)
        ax.get_yticklabels()[0].set_visible(False)

plt.show()

fig.savefig("err_" + nucleon + "_q.pdf", bbox_inches="tight")

#### Figure 3: 3He

In [ ]:
a4_width = 8.3 * 0.9

nucleon = "he3"

fig = plt.figure(dpi=500, figsize=(a4_width, a4_width / 3.5 * 4))

cont[nucleon].plot_cut_dep(
    fig,
    ylabels=d_labels,
    legend=False,
    NQ=3,
    band=["n2lo", "n3lo", "n4lo"],
    band_type="average",
)
# cont[nucleon].plot_q_dep(fig, ylabels=labels[nucleon], legend=True)


plt.subplots_adjust(wspace=0.02, hspace=0.02 / 3.2 * 3)


axs = fig.get_axes()

ax = axs[1]
# ax.set_ylim([-8,1])

# ax = axs[6]
# ax.set_ylim([-20,30])

# leg = axs[8].get_legend()
# leg.remove()

handles, plt_labels = ax.get_legend_handles_labels()
by_label = dict(zip(plt_labels, handles))
by_label_sorted = OrderedDict()
for key in [u"NLO", u"N$^{2}$LO", u"N$^{3}$LO", u"N$^{4}$LO"]:
    by_label_sorted[key] = by_label[key]

ax = axs[0]
leg = ax.legend(
    by_label_sorted.values(),
    by_label_sorted.keys(),
    numpoints=1,
    loc="best",
    frameon=True,
)

ax = axs[10]
ax.set_ylim([-12, 17])
if nucleon == "he3":
    ax.set_ylim([-5, 5])

ax = axs[7]
ax.set_ylim([-8, 1])

for ax in axs:
    ytick_labels = ax.get_yticklabels()
    if ytick_labels:
        ax.get_yticklabels()[-1].set_visible(False)
        ax.get_yticklabels()[0].set_visible(False)

plt.show()

fig.savefig("err_" + nucleon + "_q.pdf", bbox_inches="tight")

### Figure 4

In [ ]:
new_labels = [
    "$ \\left| \\mathcal{{ {F} }}   _{{q^{{(\mathrm{is})}}}}^{\\left(0 + 1\\right)} \\right|^2 $",
    "$ \\Delta^{(r)}  $ in %",
    "$ \\Delta^{(2b)} $ in %",
]

In [ ]:
fig = plt.figure(dpi=500, figsize=(a4_width, a4_width / 3.5 * 2))


for n_nuc, nucleon in enumerate(nucleons):
    cont[nucleon].plot_amplitudes(
        fig,
        ylabels=new_labels,  # labels[nucleon][1:],
        legend=False,
        band=["n2lo", "n3lo", "n4lo"],
        row=(n_nuc + 1, 2),
        band_type="best",
        skip_waves=["nijm93-noempot"],
        amp_range=[1, 2, 3],
        legend_cols=1,
    )

    print(nucleon)

plt.subplots_adjust(wspace=0.25, hspace=0.04 / 3.5 * 2)

axs = fig.get_axes()

axs[2].set_ylim([0, 7])

axs[0].plot(
    q_itp * hbarc,
    HFF["deut"].amp(q_itp) ** 2,
    "-",
    marker="s",
    label="Helm FF",
    markersize=4,
    markevery=5,
    zorder=-10,
)
axs[3].plot(
    q_itp * hbarc,
    HFF["he3"].amp(q_itp) ** 2,
    "-",
    marker="s",
    label="Helm FF",
    markersize=4,
    markevery=5,
    zorder=-10,
)

handles, plt_labels = axs[0].get_legend_handles_labels()
by_label = dict(zip(plt_labels, handles))
by_label_sorted = OrderedDict()
for key in sorted(by_label.keys()):
    by_label_sorted[key] = by_label[key]

ax = axs[2]
# leg = ax.legend(
#     by_label_sorted.values(),
#     by_label_sorted.keys(),
#     numpoints=1,
#     loc="upper left",
#     bbox_to_anchor=(1.05, 1.0),
# )

handles, plt_labels = axs[3].get_legend_handles_labels()
by_label = dict(zip(plt_labels, handles))
by_label_sorted = OrderedDict()
for key in sorted(by_label.keys()):
    by_label_sorted[key] = by_label[key]
ax = axs[5]
leg = ax.legend(
    by_label_sorted.values(),
    by_label_sorted.keys(),
    numpoints=1,
    loc="upper left",
    bbox_to_anchor=(1.05, 2.07),
)

for ax in axs:
    ax.get_yticklabels()[-1].set_visible(False)
    ax.get_yticklabels()[0].set_visible(False)

ax.get_yticklabels()[-2].set_visible(False)


axs[0].set_ylabel("$^2H$")
# axs[4].set_ylabel("$^3H$")
axs[3].set_ylabel("$^3He$")


plt.xlim([0, 190])

plt.show()

fig.savefig("conclusion_quark.pdf", bbox_inches="tight")

# Logarithmic Interference Plot

## Quarks

In [ ]:
pars = gp.parameters(normalized=True)

nucleons = ["deut", "he3"]

amps = {}
labels = {}
cont = {}

for nucleon in nucleons:
    operators = gp.get_operators(nucleon, parameters=pars)
    ams = gp.get_amplitudes(operators, parameters=pars)

    amps[nucleon] = list(ams.query("sm == 'q'")["val"])
    labels[nucleon] = list(ams.query("sm == 'q'")["label"])
    cont[nucleon] = amplitudes.OperatorPlotContainer(amps[nucleon], estimate_type=2)

In [ ]:
A = {
    "deut": 2,
    #'h3'  : 3,
    "he3": 3,
}

tex_nuc = {
    "deut": "$^2H$",
    #'h3'  : "$^3H$",
    "he3": "$^3He$",
}

In [ ]:
nucleon = "he3"

dops = gp.get_operators(nucleon, parameters=pars)
for label, op in dops.items():
    print(label)

<div class="container">
  <h2>Operator encoding table</h2>
  <p>I have listed which operator corresponds to which quantity</p>            
  <table class="table table-hover">
    <thead>
      <tr>
        <th>$ \texttt{O_type} $</th>
        <th>$\mathcal O$</th>
        <th>Magnitude</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>O1</td>
        <td>$LO$ scalar</td>
        <td>1</td>
      </tr>
      <tr>
        <td>O2</td>
        <td>$LO$ isovector</td>
        <td>5%</td>
      </tr>
      <tr>
        <td>O3</td>
        <td>$NLO$ $q^2$</td>
        <td>0 - 1%</td>
      </tr>
      <tr>
        <td>O4</td>
        <td>$NLO$ pion</td>
        <td>?</td>
      </tr>
      <tr>
        <td>O12</td>
        <td>$LO$ isoscalar-isovector</td>
        <td>1</td>
      </tr>
      <tr>
        <td>O13</td>
        <td>$NLO$ scalar-$q^2$</td>
        <td>0 - 1%</td>
      </tr>
      <tr>
        <td>O14</td>
        <td>$NLO$ scalar-pion</td>
        <td>?</td>
      </tr>
    </tbody>
  </table>
</div>

In [ ]:
cont = {}

for nucleon in nucleons:
    dops = gp.get_operators(nucleon, parameters=pars)

    o1 = dops["scalar"]
    o2 = dops["tau3"]
    o3 = dops["2-pion-q"]
    o4 = dops["scalar_q2"]

    o12 = o1 + o2
    o12 = o12.get_amp_difference(o1, relative=False).get_amp_difference(
        o2, relative=False
    )
    o13 = o1 + o3
    o13 = o13.get_amp_difference(o1, relative=False).get_amp_difference(
        o3, relative=False
    )
    o14 = o1 + o4
    o14 = o14.get_amp_difference(o1, relative=False).get_amp_difference(
        o4, relative=False
    )

    ops = OrderedDict()
    ops[r"$\left|\mathcal{F}_{q^{(\mathrm{is})   }}^{\left(0 \right)}\right|^2$"] = o1
    ops[
        r"$\left|\mathcal{F}_{q^{(\mathrm{is})   }}^{\left(0 \right)} {-} \mathcal{F}_{q^{(\mathrm{iv})   }}^{\left(0 \right)}\right|$"
    ] = o12
    ops[
        r"$\left|\mathcal{F}_{q^{(\mathrm{is})   }}^{\left(0 \right)} {-} \mathcal{F}_{q^{(\mathrm{is})},2b}^{\left(1\right)}\right|$"
    ] = o13
    ops[
        r"$\left|\mathcal{F}_{q^{(\mathrm{is})   }}^{\left(0 \right)} {-} \mathcal{F}_{q^{(\mathrm{is})}, r}^{\left(1\right)}\right|$"
    ] = o14
    ops[r"$\left|\mathcal{F}_{q^{(\mathrm{iv})   }}^{\left(1\right)}\right|^2$"] = o2
    ops[r"$\left|\mathcal{F}_{q^{(\mathrm{is})},2b}^{\left(1\right)}\right|^2$"] = o3
    ops[r"$\left|\mathcal{F}_{q^{(\mathrm{is})}, r}^{\left(1\right)}\right|^2$"] = o4

    for key, op in ops.items():
        print(key, np.sum(op.amp_frame.amp))
        if np.sum(op.amp_frame.amp) < 0:
            op.amp_frame.loc[:, "amp"] *= -1
            ops[key] = op

    cont[nucleon] = amplitudes.OperatorPlotContainer(ops.values(), estimate_type=2)

    cont[nucleon].phen_colors = sns.color_palette("hls", 8).as_hex()

    labels = ops.keys()

In [ ]:
color = sns.color_palette("dark", 5)
color.pop(3)

### Fig 6

In [ ]:
sns.set(font="Source Sans Pro", style="ticks", font_scale=2.2)
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})

fig = plt.figure(figsize=(a4_width, a4_width / 2), dpi=500)

for n_nuc, nucleon in enumerate(nucleons):
    leg = False
    if n_nuc == 1:
        leg = True
    cont[nucleon].phen_colors = color[::-1]
    cont[nucleon].plot_amplitudes(
        fig,
        ylabels=list(labels),
        amp_range=[0, 1, 2, 3],
        one_frame=True,
        band=["n2lo"],
        phenemenological=False,
        legend=leg,
        # upper_bound_only=True,
        row=(n_nuc + 1, 2),
        band_type="best",
    )
    ax = fig.gca()
    ax.set_yscale("log")
    ax.set_ylim(1.e-3, 1.2e0)
    

In [ ]:
axs = fig.get_axes()

axs[0].set_ylabel("$| \mathcal{F} \ |^2$")

for n_nuc, (nuc, ax) in enumerate(zip(nucleons, axs)):
    ax.set_yscale("log")
    ax.set_ylim([1.0e-3, 1.5])
    ax.set_xlim([0, 110])
    ax.set_title(tex_nuc[nuc])

    if n_nuc > 0:
        [el.set_visible(False) for el in ax.get_yticklabels()]

plt.subplots_adjust(wspace=0.02)

In [ ]:
plt.show()

In [ ]:
fig.savefig("logplots_quarks_n2lo.pdf", bbox_inches="tight")

## Gluons

In [ ]:
pars = gp.parameters(self_normalized=True)

nucleons = ["deut", "h3", "he3"]

amps = {}
labels = {}
cont = {}

for nucleon in nucleons:
    operators = gp.get_operators(nucleon, parameters=pars)

    ams = gp.get_amplitudes(operators, parameters=pars)

    amps[nucleon] = list(ams.query("sm == 'g'")["val"])
    labels[nucleon] = list(ams.query("sm == 'g'")["label"])
    cont[nucleon] = amplitudes.OperatorPlotContainer(amps[nucleon], estimate_type=2)

In [ ]:
A = {
    "deut": 2,
    "h3": 3,
    "he3": 3,
}

tex_nuc = {
    "deut": "$^2H$",
    "h3": "$^3H$",
    "he3": "$^3He$",
}

In [ ]:
nucleon = 'he3'

dops = gp.get_operators(nucleon, parameters=pars)
for label, op in dops.items():
    print(label)

<div class="container">
  <h2>Operator encoding table</h2>
  <p>I have listed which operator corresponds to which quantity</p>            
  <table class="table table-hover">
    <thead>
      <tr>
        <th>$ \texttt{O_type} $</th>
        <th>$\mathcal O$</th>
        <th>Magnitude</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>O1</td>
        <td>$LO$ scalar</td>
        <td>1</td>
      </tr>
      <tr>
        <td>O2</td>
        <td>$LO$ isovector</td>
        <td>5%</td>
      </tr>
      <tr>
        <td>O3</td>
        <td>$NLO$ $q^2$</td>
        <td>0 - 1%</td>
      </tr>
      <tr>
        <td>O4</td>
        <td>$NLO$ pion</td>
        <td>?</td>
      </tr>
      <tr>
        <td>O12</td>
        <td>$LO$ isoscalar-isovector</td>
        <td>1</td>
      </tr>
      <tr>
        <td>O13</td>
        <td>$NLO$ scalar-$q^2$</td>
        <td>0 - 1%</td>
      </tr>
      <tr>
        <td>O14</td>
        <td>$NLO$ scalar-pion</td>
        <td>?</td>
      </tr>
    </tbody>
  </table>
</div>

In [ ]:
cont = {}

for nucleon in nucleons:
    dops = gp.get_operators(nucleon, parameters=pars)

    o1 = dops["scalar-g"]
    o2 = dops["2-pion-g"]

    o12 = o1 + o2
    o12 = o12.get_amp_difference(o1, relative=False).get_amp_difference(
        o2, relative=False
    )

    ops = OrderedDict()
    ops[r"$\mathcal{F}_{G}^{(\mathrm{LO})}    $"] = o1
    ops[
        r"$\mathcal{F}_{G}^{(\mathrm{LO})}    $-$\mathcal{F}_{G, 2b}^{(\mathrm{N^3LO} )}$"
    ] = o12
    ops[r"$\mathcal{F}_{G, 2b}^{(\mathrm{N^3LO} )}$"] = o2

    for key, op in ops.items():
        print(key, np.sum(op.amp_frame.amp))
        if np.sum(op.amp_frame.amp) < 0:
            op.amp_frame.loc[:, "amp"] *= -1
            ops[key] = op

    cont[nucleon] = amplitudes.OperatorPlotContainer(ops.values(), estimate_type=2)

    cont[nucleon].phen_colors = sns.color_palette("hls", 8).as_hex()

    labels = ops.keys()

In [ ]:
fig = plt.figure(figsize=(a4_width, a4_width / 3), dpi=500)

for n_nuc, nucleon in enumerate(nucleons):
    leg = False
    if n_nuc == 2:
        leg = True
    cont[nucleon].phen_colors = sns.color_palette(n_colors=10)
    cont[nucleon].plot_amplitudes(
        fig,
        ylabels=labels,
        one_frame=True,
        band=["n3lo"],
        phenemenological=False,
        legend=leg,
        upper_bound_only=True,
        row=(n_nuc + 1, 3),
    )

In [ ]:
axs = fig.get_axes()

axs[0].set_ylabel("$| \mathcal{F} \ |^2 / A^2$")

for n_nuc, (nuc, ax) in enumerate(zip(nucleons, axs)):
    ax.set_yscale("log")
    ax.set_ylim([1.0e-6, 1.5])
    ax.set_xlim([0, 110])
    ax.set_title(tex_nuc[nuc])

    if n_nuc > 0:
        [el.set_visible(False) for el in ax.get_yticklabels()]

plt.subplots_adjust(wspace=0.02)

In [ ]:
fig

In [ ]:
# fig.savefig("logplots_gluons_n3lo.pdf", bbox_inches="tight")

# Cancelation Plots

## Amplitude Plot

In [ ]:
pars = gp.parameters(self_normalized=True, isoscalar_only=True)

nucleons = ["deut", "he3"]

amps = {}
labels = {}
cont = {}
r = {}
# get pre factors


for nucleon in nucleons:
    operators = gp.get_operators(nucleon, parameters=pars, ci=(1, 1, 1))

    ams = gp.get_amplitudes(operators, parameters=pars, adjust_g_q_piece=True)
    ams.set_index(["sm", "chi", "type"], inplace=True)

    glo = ams.loc[("g", "lo", "amp"), "val"]
    qlo = ams.loc[("q", "lo", "amp"), "val"]
    gnlo = ams.loc[("g", "nlo", "amp"), "val"]
    qnlo = ams.loc[("q", "nlo", "amp"), "val"]

    qlo0 = qlo.frame.loc[0, "mat"]
    glo0 = glo.frame.loc[0, "mat"]
    qnlo0 = qnlo.frame.loc[0, "mat"]

    eqs = ((qnlo0 - S("a") * glo0) / (-S("a") * glo0 + qlo0)) ** 2 - 1.5
    sol = np.sqrt(float(solve(eqs)[0]))

    # sol = -1

    r[nucleon] = sol

    print(sol)

    fact_g = -sol
    fact_q = 1  # .5 np.sqrt(.05)/10 -1

    glo *= fact_g
    qlo *= fact_q
    gnlo *= fact_g
    qnlo *= fact_q

    tlo = glo + qlo
    tnlo = qnlo + glo
    tn3lo = qnlo + gnlo

    amps[nucleon] = [tlo, tnlo, tn3lo]
    cont[nucleon] = amplitudes.OperatorPlotContainer(amps[nucleon], estimate_type=2)

In [ ]:
labels = [
    "$ {\\left| \\mathcal{{ {F} }}   _{{q^{{(\mathrm{is})}} +G  }}^{(0)} \\right|}^2 $",
    "$ \\left| \\mathcal{{ {F} }}   _{{q^{{(\mathrm{is})}} +G}}^{(0+1)} \\right|^2 $",
    "$ \\left| \\mathcal{{ {F} }}   _{{q^{{(\mathrm{is})}} +G}}^{(0+1+3)} \\right|^2 $",
]

In [ ]:
fig = plt.figure(dpi=500, figsize=(a4_width, a4_width / 2 * 2))

In [ ]:
cont["deut"].phen_colors = [
    sns.color_palette()[1],
    sns.color_palette()[0],
    sns.color_palette()[2],
]
cont["deut"].plot_amplitudes(
    fig,
    ylabels=labels,
    legend=False,
    band=["n2lo"],
    row=(2, 2, 1),
    band_type=2,
    phenemenological=False,
    one_frame=True,
    skip_waves=["nijm93-noempot"],
    amp_range=[0, 1, 2],
    legend_cols=1,
    cut_at_zero=True,
)
cont["deut"].plot_amplitudes(
    fig,
    ylabels=labels,
    legend=False,
    band=["n3lo"],
    row=(2, 2, 2),
    band_type=2,
    phenemenological=False,
    one_frame=True,
    skip_waves=["nijm93-noempot"],
    amp_range=[0, 1, 2],
    legend_cols=1,
    cut_at_zero=True,
)


cont["he3"].phen_colors = [
    sns.color_palette()[1],
    sns.color_palette()[0],
    sns.color_palette()[2],
]
cont["he3"].plot_amplitudes(
    fig,
    ylabels=labels,
    legend=False,
    band=["n2lo"],
    row=(2, 2, 3),
    band_type=2,
    phenemenological=False,
    one_frame=True,
    skip_waves=["nijm93-noempot"],
    amp_range=[0, 1, 2],
    legend_cols=1,
    cut_at_zero=True,
)
cont["he3"].plot_amplitudes(
    fig,
    ylabels=labels,
    legend=False,
    band=["n4lo"],
    row=(2, 2, 4),
    band_type=2,
    phenemenological=False,
    one_frame=True,
    skip_waves=["nijm93-noempot"],
    amp_range=[0, 1, 2],
    legend_cols=1,
    cut_at_zero=True,
)

In [ ]:
axs = fig.get_axes()

axs[0].set_ylabel("$^2H$\n $r = {r:1.4f}$".format(r=-r["deut"]))
axs[2].set_ylabel("$^3He$\n $r = {r:1.4f}$".format(r=-r["he3"]))

axs[0].set_title("N$^2$LO Waves\n")
axs[1].set_title("N$^3$LO Waves\n")

for nax, ax in enumerate(axs):
    ax.set_xlim([0, 115])

    if nax in [0, 1]:
        ax.set_ylim([0, 0.0025])
        # ax.set_ylim([2.7,4.3])
        for xticklabel in ax.get_xticklabels():
            xticklabel.set_visible(False)
        ax.set_xlabel("")

    if nax in [2, 3]:
        # ax.set_ylim([2.7,4.1])
        ax.set_ylim([0, 0.0001])
    if nax in [1, 3]:
        for yticklabel in ax.get_yticklabels():
            yticklabel.set_visible(False)


axs[2].get_yticklabels()[-1].set_visible(False)


axs[3].legend(loc="best", frameon=True)

plt.subplots_adjust(wspace=0.02, hspace=0.02 / 2 * 2)

## Fig 7

In [ ]:
fig

In [ ]:
fig.savefig("fine_tuning_50_mg.pdf", bbox_inches="tight")